<a href="https://colab.research.google.com/github/AndyChiangSH/SITCON-2022/blob/main/Colab%5C%5BSITCON_2022%5D_40%E5%88%86%E9%90%98%E7%B0%A1%E5%96%AE%E8%81%8A%E8%81%8ANLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [SITCON 2022] 40分鐘簡單聊聊NLP

這是 **40分鐘簡單聊聊NLP** 的範例程式碼。

作者：江尚軒 (Andy Chiang)  
最後修改日期：8/14  
閱讀時間：17 分鐘


## Before you start...

1. 請先點 **檔案 >> 在雲端硬碟中儲存副本**
2. 然後切換到你自己的副本
3. 接著點 **執行階段 >> 變更執行階段類型**
4. 硬體加速器選 **GPU**
5. 點擊右上角 **連線** 就可以開始執行了!

## Install packages

因為 Colab 內建沒有 transformers 和 datasets 套件，所以要先用底下的指令下載。

(如果出現紅字，是因為套件版本有衝突，重跑一次就好了)

In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Pipeline

使用 `pipeline()` 是完成NLP任務最簡單的方式。

### 各種NLP任務

首先，引用 `pipeline()` 並指定任務為**情感分析 (sentiment-analysis)**，它會自動幫你抓最好的模型下來。

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

然後就隨便給模型一句話 "This movie is interesting."。 你看! 它就能正確分析出這句話是正面的，而且分數有0.99。

In [ ]:
classifier("This movie is interesting!")

[{'label': 'POSITIVE', 'score': 0.99983811378479}]

換給它 "This movie is boring."，也能正確分析這句話是負面的。

In [ ]:
classifier("This movie is boring!")

[{'label': 'NEGATIVE', 'score': 0.9998012185096741}]

可見使用 `pipeline()` 就是如此簡單，只要兩三行程式碼就完成了，你甚至不需要了解資料預處理或模型架構! 因為 transformers 都已經幫你封裝好了。

接著，我們換做別的任務。

我們先試試看**文本生成 (text-generation)**。

In [ ]:
generator = pipeline("text-generation")

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

給模型三隻小豬的第一句，看看它會生成什麼東西出來。

生成的結果雖然很荒謬，但至少還看起來滿流暢的，對吧?

In [ ]:
generator("Once upon a time there were three little pigs.")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Once upon a time there were three little pigs. She was pregnant with two, one her father and one his mother. The baby boy turned five months old on the day of the baby's birth. He and his mother got divorced. He had to"}]

最後試試看**問答 (question-answering)**。

In [ ]:
qa = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

給模型一段有關哈利波特的介紹，然後問它哈利波特的作者是誰? 它真的就回答J. K. Rowling!

In [ ]:
context = """
Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. 
The novels chronicle the lives of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley, 
all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns 
Harry's struggle against Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard 
governing body known as the Ministry of Magic and subjugate all wizards and Muggles (non-magical people).
"""
question = "Who is the author of Harry Potter?"

qa(question, context)

{'answer': 'J. K. Rowling',
 'end': 88,
 'score': 0.9453211426734924,
 'start': 75}

### 使用自訂模型

剛才是使用預設的模型，其實也可以使用自己想要的模型。

回到**情感分析 (sentiment-analysis)** 的任務，剛才是英文的，現在想用中文的要怎麼做呢?

1. 首先到 Hugging Face 的 [models](https://huggingface.co/models) 頁面，右邊會看到許多模型，左邊則有篩選標籤。
2. 因為情感分析算是一種文本分類的任務，所以要選 **Text Classification**。
3. 再來，語言選擇 **Chinese**。
4. 找一個情感分析的模型來用，我選 **IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment** 當作例子
5. 我們要使用此模型，必須用到 **AutoModelForSequenceClassification** 和  **AutoTokenizer** 這兩個物件，稍微介紹一下：
    - **AutoModelForSequenceClassification**：指定任務的模型，像這裡就是 SequenceClassification。如果不確定模型的架構是哪一種 (如BERT、RoBERTa...)，就使用 AutoModel ，套件會自動幫你決定。
    - **AutoTokenizer**：因為模型不可能直接吃文字，所以 Tokenizer 的功能是把句子切成一個個單字，然後轉成對應的 token。
6. 下載模型需要等待一下。


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment")
tokenizer = AutoTokenizer.from_pretrained("IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment")

Downloading:   0%|          | 0.00/749 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/390M [00:00<?, ?B/s]

和剛才一樣，只是 model 和 tokenizer 的部分改成自訂的

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

然後就可以用啦!

In [ ]:
classifier("這部電影真有趣!")

[{'label': 'Positive', 'score': 0.7309864163398743}]

In [ ]:
classifier("這部電影真無聊!")

[{'label': 'Negative', 'score': 0.9995669722557068}]

## Data preprocessing

接下來，我們試試看自己 Fine-tune 一個情感分析模型。但在這之前，必須先做資料預處理。

我們利用 Hugging Face 的 **datasets** 套件，下載 [**yelp_review_ful**l](https://huggingface.co/datasets/yelp_review_full) 這個資料集。資料來源是 Yelp 網站上用戶對於各店家的評價。

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

取其中一筆資料來看，可以發現有兩個欄位，text 是用戶評論，label 則是評分 (0\~4 代表 1\~5 顆星)

In [ ]:
dataset["train"][0]

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

然後看一下 train 和 test 資料集的大小

In [ ]:
print(len(dataset["train"]))
print(len(dataset["test"]))

650000
50000


因為資料集太大會訓練很久，所以我 train 和 test 的資料只取4000和1000就好。

In [ ]:
dataset["train"] = dataset["train"].shuffle().select(range(4000))
dataset["test"] = dataset["test"].shuffle().select(range(1000))

一樣會用到剛才提到的 **AutoTokenizer**，用來將文字轉成數字。

**bert-base-cased** 則是待會要用到的模型名稱。

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

然後，我們寫一個 function，輸入是文字，輸出是處理好的 token。

tokenizer 這裡多加了 **padding** 和 **truncation** 這兩個參數。這是為了讓模型的輸入長度相同，太短的句子會補上 padding，太長的句子則截斷。

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

然後使用 `map()` 將資料集的文字都轉為 token。

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Parameter 'function'=<function tokenize_function at 0x7f92e48eb950> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Fine-tune a model

資料處理完後，就可以來 Fine-tune 模型了。

首先，我們要用 **bert-base-cased** 這個模型，這就是最原始、完全沒有 Fine-tune 過的 BERT 模型。`num_labels=5` 表示有標籤有5類 (0~4)。

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

使用 `load_metric("accuracy")` 引用正確率作為衡量指標。

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

一樣寫個 function，輸入預測結果和正確標籤，計算正確率並回傳結果。

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

引用 **TrainingArguments** 作為訓練的超參數，`output_dir="test_trainer"` 表示存檔名稱，`evaluation_strategy="epoch"` 則表示每一個 epoch 就會衡量一次正確率，其餘超參數預設就好。

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

使用 **Trainer** 物件，將剛剛的一堆東西丟進去...

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

呼叫 `trainer.train()`，然後就結束了。就會自動開始訓練模型了，是不是很簡單啊!?

(4000筆 train data 需要訓練大約22分鐘，正確率大約60%。雖然看起來不高，但如果使用更多訓練資料，正確率應該會再更高。)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1500


Epoch,Training Loss,Validation Loss,Accuracy
1,1.307200,0.988094,0.577000
2,0.936900,0.970908,0.596000
3,0.621400,1.052324,0.606000


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num 

TrainOutput(global_step=1500, training_loss=0.9551491088867188, metrics={'train_runtime': 1340.1922, 'train_samples_per_second': 8.954, 'train_steps_per_second': 1.119, 'total_flos': 3157417709568000.0, 'train_loss': 0.9551491088867188, 'epoch': 3.0})

## Save a model

訓練完後，可以使用 `save_model()` 這個函數將模型下載下來，模型會下載在 Colab 的虛擬機器中。

In [ ]:
trainer.save_model("./test_model")

Saving model checkpoint to ./test_model
Configuration saved in ./test_model/config.json
Model weights saved in ./test_model/pytorch_model.bin


使用 `from_pretrained()` 就可以載入模型來使用。

In [ ]:
from transformers import AutoModelForSequenceClassification

pt_model = AutoModelForSequenceClassification.from_pretrained("./test_model")

loading configuration file ./test_model/config.json
Model config BertConfig {
  "_name_or_path": "./test_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": tr

transformers 還有一個很強大的功能，就在於 Pytorch 和 TensorFlow 的模型可以輕易轉換。

剛才下載的是 Pytorch 的模型，但只要在使用 TensorFlow 的模型載入時加上 `from_pt=True`，就可以轉換成功了!

In [ ]:
from transformers import TFAutoModelForSequenceClassification

tf_model = TFAutoModelForSequenceClassification.from_pretrained("./test_model", from_pt=True)

loading configuration file ./test_model/config.json
Model config BertConfig {
  "_name_or_path": "./test_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": tr

## Share a model

如果你想把 Fine-tune 好的模型分享給大家，首先你需要下載 **huggingface_hub** 套件。

In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 101 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 28.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


下一步要登入，執行後會出現如同下面的提示，要求你輸入一個 access token。

![](https://i.imgur.com/2a0zns2.png)

點擊提示裡的[連結](https://huggingface.co/settings/tokens)，會傳到 Hugging Face 的網站，新增一個權限是 **write** 的 token。

![](https://i.imgur.com/614QPsM.png)

然後將這個 token 填入文字框內並按 login，就能成功登入了。如果沒有帳號必須先註冊一個 (怎麼註冊應該不用講了吧)。

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


再來就只要把剛才 Fine-tune 好的模型 push 到 Hugging Face 上就行了，它會自動地幫你創立新的 repo。

In [ ]:
pt_model.push_to_hub("my-test-model")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:43: FutureWarning: Pass token='my-test-model' as keyword args. From version 0.8 passing these as positional arguments will result in an error
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:644: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/AndyChiang/my-test-model into local empty directory.
Configuration saved in my-test-model/config.json
Model weights saved in my-test-model/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.34k/413M [00:00<?, ?B/s]

To https://huggingface.co/AndyChiang/my-test-model
   da541a5..9035379  main -> main



'https://huggingface.co/AndyChiang/my-test-model/commit/9035379b12a638eeb8e54d38cdc3f47474e03b8d'

雖然說 transformers 可以自動轉換 Pytorch 和 TensorFlow 的模型，但會比較花時間。所以為了使用者方便，基本上建議你將兩種模型都 push 上去比較好。

In [ ]:
tf_model.push_to_hub("my-test-model")

Configuration saved in my-test-model/config.json
Model weights saved in my-test-model/tf_model.h5


Upload file tf_model.h5:   0%|          | 3.34k/413M [00:00<?, ?B/s]

To https://huggingface.co/AndyChiang/my-test-model
   9c4ac9f..e6392a9  main -> main



'https://huggingface.co/AndyChiang/my-test-model/commit/e6392a97d572dd50121bb398803a008dc230bb60'

也把剛才用的 tokenizer 給 push 上去。

In [ ]:
tokenizer.push_to_hub("my-test-model")

tokenizer config file saved in my-test-model/tokenizer_config.json
Special tokens file saved in my-test-model/special_tokens_map.json
To https://huggingface.co/AndyChiang/my-test-model
   9035379..9c4ac9f  main -> main



'https://huggingface.co/AndyChiang/my-test-model/commit/9c4ac9fee5b0a98c227c918bef6280cc1d695c1a'

以後就可以下載自己的模型來用啦! 名稱是 `<user name>/<model name>`

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

my_model = AutoModelForSequenceClassification.from_pretrained("AndyChiang/my-test-model")
my_tokenizer = AutoTokenizer.from_pretrained("AndyChiang/my-test-model")

loading configuration file https://huggingface.co/AndyChiang/my-test-model/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/151c4d8356041ed68239362dc8dc5395537e3fd7a973ecb205a1e548296a4052.face7811affdda36ddf9cd32db1c2c3e6106f0b3eaa854d9d00c2f75900f076d
Model config BertConfig {
  "_name_or_path": "AndyChiang/my-test-model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
 

用自己的模型來測試情感分析。

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=my_model, tokenizer=my_tokenizer)

結果還不錯!

In [ ]:
classifier("This restaurant is great! The food there is delicious, too.")

[{'label': 'LABEL_4', 'score': 0.9106563329696655}]

最後提一下網站的部分，你可以到 Hugging Face 的網站上查看自己的模型，以[我的模型](https://huggingface.co/AndyChiang/my-test-model)為例。你可以編輯 **model card** (類似 GitHub README)，寫上一些關於模型的資訊，也可以用 **Hosted inference API**
來做些簡單的測試。

![](https://i.imgur.com/hmprtP2.png)


## Summary



恭喜你看完整個教學了! 相信你現在已經有基礎的 NLP 知識，也可以自己 Fine-tune 模型了! 不過這還只是基礎而已，NLP 還有很多東西沒講到，但剩下的就留給有興趣的讀者自行研究吧! 也歡迎分享你的學習心得~

哦對了! 很推薦對 NLP 有興趣的人去看 [Transformers 的 document](https://huggingface.co/docs/transformers/index)，內容非常豐富且完整。此教學大部分也是參考這個 document 的哦🤗